 读取训练文件

In [1]:
f = open('链路预测数据集/FB15k/freebase_mtr100_mte100-train.txt')
data = f.read()
triples = data.split('\n')
triples = triples[:483142]

totals = list([entity_and_relation for triple in triples for entity_and_relation in triple.split('\t')])
print('三元组数量',len(triples))
print('实体总数',len(set(totals)))
#嵌入的总数为16296

三元组数量 483142
实体总数 16296


生成头实体集合 尾实体集合 关系集合

In [2]:
head_entities = list([triple.split('\t')[0] for triple in triples])
relations =  list([triple.split('\t')[1] for triple in triples])
tail_entities =  list([triple.split('\t')[2] for triple in triples])

total_entities = head_entities + tail_entities

print('实体数',len(set(head_entities + tail_entities)))
print('关系数',len(set(relations)))

实体数 14951
关系数 1345


生成负采样三元组

In [3]:
import random

# 生成负采样的三元组
invalid_head_entities = []
invalid_relations = []
invalid_tail_entities = []

for i in range(len(relations)):
    random_int = random.randint(0,len(head_entities)-1)
    random_int1 = random.randint(0,len(head_entities)-1)
    
    # 不同时替换头尾实体
    if i%2 ==0:
        invalid_head_entities.append(head_entities[i])
        invalid_relations.append(relations[i])
        invalid_tail_entities.append(tail_entities[random_int1])
    else:
        invalid_head_entities.append(head_entities[random_int])
        invalid_relations.append(relations[i])
        invalid_tail_entities.append(tail_entities[i])
#     invalid_head_entities.append(head_entities[random_int])
#     invalid_relations.append(relation)
#     invalid_tail_entities.append(tail_entities[random_int1])

print(head_entities[123456],relations[123456],tail_entities[123456])
print(head_entities[12345],relations[12345],tail_entities[12345])
print(head_entities[12234],relations[12234],tail_entities[12234])

print(invalid_head_entities[123456],invalid_relations[123456],invalid_tail_entities[123456])
print(invalid_head_entities[12345],invalid_relations[12345],invalid_tail_entities[12345])
print(invalid_head_entities[12234],invalid_relations[12234],invalid_tail_entities[12234])

/m/02w2bc /education/educational_institution/students_graduates./education/education/major_field_of_study /m/0g26h
/m/0kpzy /location/us_county/county_seat /m/0dc95
/m/081pw /military/military_conflict/combatants./military/military_combatant_group/combatants /m/0193qj
/m/02w2bc /education/educational_institution/students_graduates./education/education/major_field_of_study /m/03mh_tp
/m/05kkh /location/us_county/county_seat /m/0dc95
/m/081pw /military/military_conflict/combatants./military/military_combatant_group/combatants /m/01hqhm


对实体编码

In [4]:
# 为每个实体/关系分配倒排索引
# 根据 名称 查找 id
reverse_index = dict([key,index] for index,key in enumerate(set(totals)))
print('索引长度',len(reverse_index))

#数据标签化

head_entities_encode = list([reverse_index[entry] for entry in head_entities])
relations_encode = list([reverse_index[relation] for relation in relations])
tail_entities_encode = list([reverse_index[entry] for entry in tail_entities])

invalid_head_entities_encode = list([reverse_index[entry] for entry in invalid_head_entities])
invalid_relations_encode = list([reverse_index[relation] for relation in invalid_relations])
invalid_tail_entities_encode = list([reverse_index[entry] for entry in invalid_tail_entities])

print(head_entities_encode[123456],relations_encode[123456],tail_entities_encode[123456])
print(head_entities_encode[12345],relations_encode[12345],tail_entities_encode[12345])
print(head_entities_encode[12234],relations_encode[12234],tail_entities_encode[12234])

print(invalid_head_entities_encode[123456],invalid_relations_encode[123456],invalid_tail_entities_encode[123456])
print(invalid_head_entities_encode[12345],invalid_relations_encode[12345],invalid_tail_entities_encode[12345])
print(invalid_head_entities_encode[12234],invalid_relations_encode[12234],invalid_tail_entities_encode[12234])

索引长度 16296
10290 10568 8488
9475 15129 6499
14285 7140 2653
10290 10568 9020
9565 15129 6499
14285 7140 13143


组合数据

In [5]:
# 编码后的数据
golden_data = list(zip(head_entities_encode,relations_encode,tail_entities_encode))
invalid_data = list(zip(invalid_head_entities_encode,invalid_relations_encode,invalid_tail_entities_encode))
    
print(golden_data[123456])
print(invalid_data[123456])

print('三元组数量',len(golden_data))
print('三元组数量',len(invalid_data))

(10290, 10568, 8488)
(10290, 10568, 9020)
三元组数量 483142
三元组数量 483142


获取数据

In [6]:
keep = 0 
def nextbatch(batchsize=400):
    global keep
    start = keep
    end = start + batchsize
    if end > len(golden_data):
        end = end % len(golden_data)
        keep = end
        return golden_data[start:]+golden_data[:end],invalid_data[start:]+invalid_data[:end]
    else:
        keep = end
        return golden_data[start:end],invalid_data[start:end]
    
# golden_data = golden_data[:10]
# invalid_data = invalid_data[:10]  

# print(nextbatch(3)[0])
# print(nextbatch(5)[0])
# print(nextbatch(4)[0])
# print(nextbatch(4)[0])
# print(nextbatch(4)[0])

In [7]:
import tensorflow as tf
import math
import time

vocabulary_size = 16296
embedding_size = 50
epochs_num = 200
batch_size = 401
margin = 1
learning_rate = 1e-1

#定义模型
def l1_energy(batch):
    return tf.reduce_sum(tf.abs(batch[:, 0, :] + batch[:, 1, :] - batch[:, 2, :]), 1)

embedding_table = tf.Variable(tf.truncated_normal([vocabulary_size, embedding_size], stddev=1.0 / math.sqrt(embedding_size)))

pos_triples_batch = tf.placeholder(tf.int32, shape=[batch_size, 3])
neg_triples_batch = tf.placeholder(tf.int32, shape=[batch_size, 3])
predict_triples_batch = tf.placeholder(tf.int32,shape=[1,1])

pos_embed_batch = tf.nn.embedding_lookup(embedding_table, pos_triples_batch)
neg_embed_batch = tf.nn.embedding_lookup(embedding_table, neg_triples_batch)
predict_embed_batch = tf.nn.embedding_lookup(embedding_table, predict_triples_batch)

p_loss = l1_energy(pos_embed_batch)
n_loss = l1_energy(neg_embed_batch)

loss = tf.reduce_sum(tf.nn.relu(margin + p_loss - n_loss))
optimizer = tf.train.AdagradOptimizer(learning_rate).minimize(loss)

sess = tf.Session()
sess.run(tf.initialize_all_variables())

for step in range(epochs_num):
    time_start = time.time() #开始计时
    loss_total = 0
    print('start:',keep)
    
    for i in range(len(golden_data)//batch_size + 1):
        
        golden_batch,invalid_batch = nextbatch(batchsize=batch_size)    
        feed_dict = {pos_triples_batch: golden_batch, neg_triples_batch: invalid_batch}
        loss_val, _ = sess.run([loss, optimizer], feed_dict=feed_dict)
        loss_total += loss_val
        
    print("\rstep %d, loss_val %f" % (step, loss_total/(len(golden_data)//batch_size + 1)))
    time_end = time.time()    #结束计时
    sum_t= time_end - time_start   #运行所花时间
    print('time cost', sum_t, 's')

Instructions for updating:
Use `tf.global_variables_initializer` instead.
start: 0
step 0, loss_val 183.860747
time cost 3.6093409061431885 s
start: 63
step 1, loss_val 22.910184
time cost 3.4926540851593018 s
start: 126
step 2, loss_val 6.748709
time cost 3.1196513175964355 s
start: 189
step 3, loss_val 2.603784
time cost 2.99302339553833 s
start: 252
step 4, loss_val 1.235208
time cost 3.2911648750305176 s
start: 315
step 5, loss_val 0.720106
time cost 3.330087184906006 s
start: 378
step 6, loss_val 0.513691
time cost 2.709749698638916 s
start: 441
step 7, loss_val 0.411646
time cost 3.6033577919006348 s
start: 504
step 8, loss_val 0.355325
time cost 3.40189790725708 s
start: 567
step 9, loss_val 0.325576
time cost 3.0807571411132812 s
start: 630
step 10, loss_val 0.311744
time cost 3.027897596359253 s
start: 693
step 11, loss_val 0.300695
time cost 2.6957876682281494 s
start: 756
step 12, loss_val 0.291384
time cost 2.869321823120117 s
start: 819
step 13, loss_val 0.284499
time cost

step 114, loss_val 0.272199
time cost 2.8025004863739014 s
start: 7245
step 115, loss_val 0.272199
time cost 2.790534019470215 s
start: 7308
step 116, loss_val 0.272199
time cost 2.794124126434326 s
start: 7371
step 117, loss_val 0.273029
time cost 2.8005189895629883 s
start: 7434
step 118, loss_val 0.272199
time cost 2.797503709793091 s
start: 7497
step 119, loss_val 0.272199
time cost 2.7815568447113037 s
start: 7560
step 120, loss_val 0.272199
time cost 2.9261696338653564 s
start: 7623
step 121, loss_val 0.272199
time cost 2.808486223220825 s
start: 7686
step 122, loss_val 0.272199
time cost 2.767594575881958 s
start: 7749
step 123, loss_val 0.272199
time cost 2.7636055946350098 s
start: 7812
step 124, loss_val 0.272199
time cost 2.794527053833008 s
start: 7875
step 125, loss_val 0.272199
time cost 2.7755701541900635 s
start: 7938
step 126, loss_val 0.272199
time cost 2.776569128036499 s
start: 8001
step 127, loss_val 0.272199
time cost 2.7975149154663086 s
start: 8064
step 128, los

In [8]:
total_embedding_index = []

for entity in list(reverse_index):
    encode = reverse_index[entity]
    embedding = sess.run([predict_embed_batch], feed_dict= {predict_triples_batch: [[encode]]})[0][0][0]
    total_embedding_index.append([entity,embedding])
    
TransE_index = dict(total_embedding_index)
# print(TransE_index['/m/035yzw'])

entity_embedding_index = []
for entity in list(total_entities):
    encode = reverse_index[entity]
    embedding = sess.run([predict_embed_batch], feed_dict= {predict_triples_batch: [[encode]]})[0][0][0]
    entity_embedding_index.append([entity,embedding])
    
TransE_entity_index = dict(entity_embedding_index)

sess.close()

链接预测评测代码

In [9]:
# 读入训练数据
# 先用hit@10标准测试一下
valid_file = open('链路预测数据集/FB15k/freebase_mtr100_mte100-valid.txt')
valid_data = valid_file.read()
valid_triples = valid_data.split('\n')[:50000]

#生成验证集实体和关系集合
valid_head_entities = list([triple.split('\t')[0] for triple in valid_triples])
valid_relations =  list([triple.split('\t')[1] for triple in valid_triples])
valid_tail_entities =  list([triple.split('\t')[2] for triple in valid_triples])

In [ ]:
import numpy as np

hit10 = 0
hit50 = 0
hit1000 = 0
num = 0

def takeFirst(elem):
    return elem[0]

# for head,relation,tail in zip(head_entities[:10],relations[:10],tail_entities[:10]):
for head,relation,tail in zip(valid_head_entities,valid_relations,valid_tail_entities):
    num = num+1
    
    print(head,relation,tail)
    #　头实体嵌入
    valid_head_embedding = TransE_index[head]
    #　关系的嵌入
    valid_relation_embedding = TransE_index[relation]
    #　头尾实体加和
    head_and_relation = valid_head_embedding + valid_relation_embedding
    # 预测一个尾实体
    predict_tail_embedding = head_and_relation
    
    #　与所有的实体计算距离
    distance_list = []
    
    # 遍历TransE嵌入中的所有实体
    for index,entity in enumerate(list(TransE_entity_index)):
        
        #　与TransE中的所有实体进行比较
        embedding = TransE_index[entity]
        
        # 二阶距离
        distance = np.linalg.norm(predict_tail_embedding - embedding)
        distance_list.append([distance,entity])

        # 对所有距离进行排序
        distance_list.sort(key=takeFirst)

    print(distance_list[0])

    predict_1000_list = list([entity[1] for entity in distance_list])[:1000]
    predict_50_list = list([entity[1] for entity in distance_list])[:50]
    predict_10_list = list([entity[1] for entity in distance_list])[:10]

    if tail in predict_1000_list:
        hit1000 = hit1000+1
#         print('hit@1000yes')
#     else:
#         print('hit@1000no')
        
    if tail in predict_50_list:
        hit50 = hit50+1
#         print('hit@50yes')
#     else:
#         print('hit@50no')
        
    if tail in predict_10_list:
        hit10 = hit10 +1
#         print('hit@10yes')
#     else:
#         print('hit@10no')
        
    print('hit',hit10/num,hit50/num,hit1000/num,num)

/m/07pd_j /film/film/genre /m/02l7c8
[2.171723, '/m/0bkbm']
hit 0.0 1.0 1.0 1
/m/06wxw /location/location/time_zones /m/02fqwt
[2.198836, '/m/06wxw']
hit 0.5 1.0 1.0 2
/m/0d4fqn /award/award_winner/awards_won./award/award_honor/award_winner /m/03wh8kl
[0.9404455, '/m/0d4fqn']
hit 0.3333333333333333 0.6666666666666666 0.6666666666666666 3
/m/07kcvl /american_football/football_team/historical_roster./american_football/football_historical_roster_position/position_s /m/0bgv8y
[1.8774172, '/m/07kcvl']
hit 0.25 0.5 0.75 4
/m/012201 /film/music_contributor/film /m/0ckrnn
[1.7456454, '/m/0564x']
hit 0.2 0.4 0.8 5
/m/018gz8 /people/profession/people_with_this_profession /m/01520h
[1.8995486, '/m/0qkj7']
hit 0.16666666666666666 0.3333333333333333 0.8333333333333334 6
/m/02x17c2 /award/award_category/nominees./award/award_nomination/award_nominee /m/0x3b7
[2.1223142, '/m/020fgy']
hit 0.14285714285714285 0.2857142857142857 0.7142857142857143 7
/m/03swmf /people/person/education./education/educatio

hit 0.21739130434782608 0.3333333333333333 0.6086956521739131 69
/m/051vz /baseball/baseball_team/current_roster./sports/sports_team_roster/position /m/028c_8
[1.4145249, '/m/051vz']
hit 0.21428571428571427 0.32857142857142857 0.6 70
/m/05t0_2v /award/award_winning_work/awards_won./award/award_honor/award_winner /m/03q3sy
[1.5026069, '/m/0gy6z9']
hit 0.2112676056338028 0.323943661971831 0.6056338028169014 71
/m/0kvsb /people/person/profession /m/01d30f
[2.4606054, '/m/02hv44_']
hit 0.20833333333333334 0.3194444444444444 0.6111111111111112 72
/m/02y9ln /soccer/football_player/position_s /m/02sdk9v
[1.5673085, '/m/02y9ln']
hit 0.2054794520547945 0.3150684931506849 0.6164383561643836 73
/m/02r96rf /film/film_job/films_with_this_crew_job./film/film_crew_gig/film /m/07gp9
[2.0621848, '/m/02vqsll']
hit 0.20270270270270271 0.3108108108108108 0.6216216216216216 74
/m/0283d /music/genre/parent_genre /m/06cqb
[1.4362864, '/m/0283d']
hit 0.2 0.30666666666666664 0.6266666666666667 75
/m/0cv9fc /fi

hit 0.18518518518518517 0.2962962962962963 0.6666666666666666 135
/m/0l15bq /music/performance_role/track_performances./music/track_contribution/role /m/01w4dy
[1.7337638, '/m/0l15bq']
hit 0.19117647058823528 0.3014705882352941 0.6691176470588235 136
/m/0k2cb /film/film/country /m/09c7w0
[2.282791, '/m/0jm6n']
hit 0.1897810218978102 0.30656934306569344 0.6715328467153284 137
/m/04zpv /food/food/nutrients./food/nutrition_fact/nutrient /m/0h1yy
[1.7860074, '/m/05v_8y']
hit 0.18840579710144928 0.3115942028985507 0.6739130434782609 138
/m/06tp4h /people/person/gender /m/02zsn
[2.0687215, '/m/05zppz']
hit 0.19424460431654678 0.31654676258992803 0.6762589928057554 139
/m/025_d68 /fictional_universe/fictional_universe/works_set_here /m/08cx5g
[1.205349, '/m/025_d68']
hit 0.19285714285714287 0.3142857142857143 0.6714285714285714 140
/m/076lxv /award/award_winner/awards_won./award/award_honor/ceremony /m/0c53zb
[2.0526588, '/m/0dznvw']
hit 0.19148936170212766 0.3120567375886525 0.67375886524822

hit 0.1935483870967742 0.3225806451612903 0.6827956989247311 186
/m/0nbcg /people/profession/people_with_this_profession /m/02rf51g
[1.9233112, '/m/02wd48']
hit 0.1925133689839572 0.32085561497326204 0.679144385026738 187
/m/018wdw /award/award_category/nominees./award/award_nomination/nominated_for /m/0gwjw0c
[2.2063355, '/m/072r5v']
hit 0.19148936170212766 0.3191489361702128 0.6808510638297872 188
/m/02wcx8c /film/actor/film./film/performance/film /m/02ppg1r
[1.8766074, '/m/0c1sgd3']
hit 0.19047619047619047 0.31746031746031744 0.6772486772486772 189
/m/095zvfg /film/film_crewmember/films_crewed./film/film_crew_gig/film_crew_role /m/02vs3x5
[1.8073581, '/m/095zvfg']
hit 0.18947368421052632 0.32105263157894737 0.6789473684210526 190
/m/08q3s0 /award/award_winner/awards_won./award/award_honor/award_winner /m/0h53p1
[0.9404455, '/m/08q3s0']
hit 0.18848167539267016 0.3193717277486911 0.680628272251309 191
/m/04mcw4 /film/film/starring./film/performance/actor /m/02yxwd
[1.8482232, '/m/0m2w

hit 0.17446808510638298 0.3148936170212766 0.676595744680851 235
/m/091tgz /base/marchmadness/ncaa_basketball_team/ncaa_tournament_seeds./base/marchmadness/ncaa_tournament_seed/tournament /m/0b_6xf
[1.6524469, '/m/091tgz']
hit 0.17372881355932204 0.3135593220338983 0.673728813559322 236
/m/01_3rn /military/military_conflict/combatants./military/military_combatant_group/combatants /m/01tdpv
[1.8825125, '/m/0gd70t']
hit 0.1729957805907173 0.31223628691983124 0.6708860759493671 237
/m/033tln /people/person/places_lived./people/place_lived/location /m/0r172
[1.9194825, '/m/02_286']
hit